# Character encodings
## 1. Tutoriel

In [1]:
# modules we'll use
import pandas as pd
import numpy as np

# helpful character encoding module
import chardet

# set seed for reproducibility
np.random.seed(0)

### Qu'est-ce que l'encodage ?

**L'encodage de caractères** est un ensemble de règles spécifiques pour le mappage de chaînes d'octets binaires brutes (qui ressemblent à ceci : 0110100001101001) en caractères qui composent un texte lisible par l'homme (comme « bonjour »). Il existe de nombreux encodages différents, et si vous essayez de lire un texte avec un encodage différent de celui dans lequel il a été écrit à l'origine, vous vous retrouvez avec un texte brouillé appelé "mojibake" (dit comme mo-gee-bah-kay). Voici un exemple de mojibake :
æ–‡å—åŒ–ã??

Vous pourriez également vous retrouver avec des caractères « inconnus ». Il y a ce qui est imprimé lorsqu'il n'y a pas de mappage entre un octet particulier et un caractère dans l'encodage que vous utilisez pour lire votre chaîne d'octets et ils ressemblent à ceci :
����������

Les incompatibilités d'encodage de caractères sont moins courantes aujourd'hui qu'auparavant, mais c'est certainement toujours un problème. Il existe de nombreux encodages de caractères différents, mais le principal que vous devez connaître est UTF-8.

> UTF-8 is **the** standard text encoding. All Python code is in UTF-8 and, ideally, all your data should be as well. It's when things aren't in UTF-8 that you run into trouble.

Il était assez difficile de gérer les encodages en Python 2, mais heureusement, en Python 3, c'est beaucoup plus simple. Vous rencontrerez deux types de données principaux lorsque vous travaillerez avec du texte dans Python 3. L'un est la chaîne, qui est le texte par défaut.

In [2]:
# start with a string
before = "This is the euro symbol: €"

# check to see what datatype it is
type(before)

str

Les autres données sont le type de données [bytes](https://docs.python.org/3.1/library/functions.html#bytes), qui est une séquence d'entiers. Vous pouvez convertir une chaîne en octets en spécifiant dans quel encodage elle se trouve :

In [3]:
# encode it to a different encoding, replacing characters that raise errors
after = before.encode("utf-8", errors="replace")

# check the type
type(after)

bytes

Si vous regardez un objet bytes, vous verrez qu'il a un b devant lui, puis peut-être du texte après. C'est parce que les octets sont imprimés comme s'il s'agissait de caractères codés en ASCII. (ASCII est un ancien codage de caractères qui ne fonctionne pas vraiment pour écrire une langue autre que l'anglais.) Ici, vous pouvez voir que notre symbole de l'euro a été remplacé par un mojibake qui ressemble à "\xe2\x82\xac" lorsqu'il est imprimé comme s'il s'agissait d'une chaîne ASCII.

In [4]:
# take a look at what the bytes look like
after

b'This is the euro symbol: \xe2\x82\xac'

Lorsque nous reconvertissons nos octets en une chaîne avec l'encodage correct, nous pouvons voir que notre texte est bien là, ce qui est génial ! :)

In [5]:
# convert it back to utf-8
print(after.decode("utf-8"))

This is the euro symbol: €


Cependant, lorsque nous essayons d'utiliser un encodage différent pour mapper nos octets dans une chaîne, nous obtenons une erreur. C'est parce que l'encodage que nous essayons d'utiliser ne sait pas quoi faire avec les octets que nous essayons de lui transmettre. Vous devez indiquer à Python l'encodage dans lequel la chaîne d'octets est réellement censée se trouver.

> Vous pouvez considérer différents encodages comme différentes manières d'enregistrer de la musique. Vous pouvez enregistrer la même musique sur un CD, une cassette ou 8 pistes. Bien que la musique puisse sembler plus ou moins la même, vous devez utiliser le bon équipement pour lire la musique à partir de chaque format d'enregistrement. Le bon décodeur est comme un lecteur de cassettes ou un lecteur de CD. Si vous essayez de lire une cassette dans un lecteur de CD, cela ne fonctionnera tout simplement pas.

In [6]:
# try to decode our bytes with the ascii encoding
print(after.decode("ascii"))

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe2 in position 25: ordinal not in range(128)

Nous pouvons également rencontrer des problèmes si nous essayons d'utiliser le mauvais encodage pour mapper une chaîne en octets. Comme je l'ai dit plus tôt, les chaînes sont UTF-8 par défaut dans Python 3, donc si nous essayons de les traiter comme si elles étaient dans un autre encodage, nous créerons des problèmes.

Par exemple, si nous essayons de convertir une chaîne en octets pour l'ASCII en utilisant `encode()`, nous pouvons demander que les octets soient ce qu'ils seraient si le texte était en ASCII. Comme notre texte n'est pas en ASCII, il y aura cependant certains caractères qu'il ne pourra pas gérer. Nous pouvons remplacer automatiquement les caractères que l'ASCII ne peut pas gérer. Si nous faisons cela, cependant, tous les caractères qui ne sont pas en ASCII seront simplement remplacés par le caractère inconnu. Ensuite, lorsque nous reconvertirons les octets en chaîne, le caractère sera remplacé par le caractère inconnu. La partie dangereuse à ce sujet est qu'il n'y a aucun moyen de dire quel personnage cela aurait dû être. Cela signifie que nous venons peut-être de rendre nos données inutilisables !

In [7]:
# start with a string
before = "This is the euro symbol: €"

# encode it to a different encoding, replacing characters that raise errors
after = before.encode("ascii", errors = "replace")

# convert it back to utf-8
print(after.decode("ascii"))

# We've lost the original underlying byte string! It's been 
# replaced with the underlying byte string for the unknown character :(

This is the euro symbol: ?


C'est mauvais et nous voulons éviter de le faire ! Il est de loin préférable de convertir tout notre texte en UTF-8 dès que possible et de le conserver dans cet encodage. Le meilleur moment pour convertir une entrée non UTF-8 en UTF-8 est lorsque vous lisez des fichiers, dont nous parlerons ensuite.

### Lecture de fichiers avec des problèmes d'encodage

La plupart des fichiers que vous rencontrerez seront probablement encodés en UTF-8. C'est ce que Python attend par défaut, donc la plupart du temps, vous ne rencontrerez pas de problèmes. Cependant, vous obtiendrez parfois une erreur comme celle-ci :

In [8]:
# try to read in a file not in UTF-8
kickstarter_2016 = pd.read_csv("../Data/data1/ks-projects-201612.csv")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x99 in position 11: invalid start byte

Notez que nous obtenons la même `UnicodeDecodeError` que lorsque nous avons essayé de décoder les octets UTF-8 comme s'ils étaient ASCII ! Cela nous indique que ce fichier n'est pas réellement UTF-8. Cependant, nous ne savons pas quel encodage il *est*. Une façon de le comprendre est d'essayer de tester un tas d'encodages de caractères différents et de voir si l'un d'entre eux fonctionne. Une meilleure façon, cependant, est d'utiliser le module chardet pour essayer de deviner automatiquement quel est le bon encodage. Ce n'est pas garanti à 100%, mais c'est généralement plus rapide que d'essayer de deviner.

Je vais juste regarder les dix mille premiers octets de ce fichier. C'est généralement suffisant pour une bonne estimation de l'encodage et c'est beaucoup plus rapide que d'essayer de regarder l'ensemble du fichier. (Surtout avec un gros fichier, cela peut être très lent.) Une autre raison de ne regarder que la première partie du fichier est que nous pouvons voir en regardant le message d'erreur que le premier problème est le 11ème caractère. Nous n'avons donc probablement besoin que de regarder la première partie du fichier pour comprendre ce qui se passe.

In [9]:
# look at the first ten thousand bytes to guess the character encoding
with open("../Data/data1/ks-projects-201801.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))

# check what the character encoding might be
print(result)

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


Donc chardet a 73% de confiance que le bon encodage est "Windows-1252". Voyons si c'est correct :

In [10]:
# read in the file with the encoding detected by chardet
kickstarter_2016 = pd.read_csv("../Data/data1/ks-projects-201612.csv", encoding='Windows-1252')

# look at the first few lines
kickstarter_2016.head()

C:\Users\GHESMOMO\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000,2015-08-11 12:12:28,0,failed,0,GB,0,NaN,NaN,NaN,NaN
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,45000,2013-01-12 00:20:50,220,failed,3,US,220,NaN,NaN,NaN,NaN
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 04:24:11,5000,2012-03-17 03:24:11,1,failed,1,US,1,NaN,NaN,NaN,NaN
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 01:00:00,19500,2015-07-04 08:35:03,1283,canceled,14,US,1283,NaN,NaN,NaN,NaN
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000,2016-02-26 13:38:27,52375,successful,224,US,52375,NaN,NaN,NaN,NaN


Oui, on dirait que chardet avait raison ! Le fichier est lu sans problème (bien que nous recevions un avertissement concernant les types de données) et lorsque nous regardons les premières lignes, cela semble bien.

> **Et si l'encodage de chardet n'est pas correct ?** Étant donné que chardet n'est fondamentalement qu'un devineur sophistiqué, il devine parfois le mauvais encodage. Une chose que vous pouvez essayer est de regarder plus ou moins le fichier et de voir si vous obtenez un résultat différent, puis essayez cela.

### Enregistrer vos fichiers avec l'encodage UTF-8

Enfin, une fois que vous vous êtes donné la peine de mettre votre fichier en UTF-8, vous voudrez probablement le garder ainsi. Le moyen le plus simple de le faire est d'enregistrer vos fichiers avec l'encodage UTF-8. La bonne nouvelle est que, puisque UTF-8 est l'encodage standard en Python, lorsque vous enregistrez un fichier, il sera enregistré en UTF-8 par défaut :

In [ ]:
# save our file (will be saved as UTF-8 by default!)
kickstarter_2016.to_csv("ks-projects-201801-utf8.csv")

Pretty easy, huh? :)

## 2. Exercice

In [ ]:
# modules we'll use
import pandas as pd
import numpy as np

# helpful character encoding module
import chardet

# set seed for reproducibility
np.random.seed(0)

What are encodings?

Vous travaillez avec un ensemble de données composé d'octets. Exécutez la cellule de code ci-dessous pour imprimer un exemple d'entrée.

In [ ]:
sample_entry = b'\xa7A\xa6n'
print(sample_entry)
print('data type:', type(sample_entry))

Vous remarquez qu'il n'utilise pas l'encodage UTF-8 standard.

Utilisez la cellule de code suivante pour créer une variable new_entry qui modifie l'encodage de "big5-tw" à "utf-8". new_entry doit avoir le type de données bytes.

In [ ]:
new_entry = ____

Lecture de fichiers avec des problèmes d'encodage

Utilisez la cellule de code ci-dessous pour lire ce fichier au chemin "../Data/data1/PoliceKillingsUS.csv".

Déterminez quel doit être l'encodage correct et lisez le fichier dans un DataFrame police_killings.

In [ ]:
# TODO: Load in the DataFrame correctly.
police_killings = ____


Enregistrer vos fichiers avec l'encodage UTF-8

Enregistrez une version de l'ensemble de données sur les meurtres de policiers au format CSV avec l'encodage UTF-8. Votre réponse sera marquée comme correcte après l'enregistrement de ce fichier.

Remarque : lorsque vous utilisez la méthode to_csv(), indiquez uniquement le nom du fichier (par exemple, "my_file.csv").

In [ ]:
# TODO: